In [ ]:
import pandas as pd
import urllib.parse as ulp

In [ ]:
fetchSize = 100 # How many records to retrieve at once from the API
offset = 0
queryPt1 = 'sys_updated_on>javascript:gs.dateGenerate('
queyrPt2 = ')^ORDERBYsys_updated_on'

In [ ]:
ulp.urlencode(mydict)

In [ ]:
qStr = {'sysparm_exclude_reference_link' : 'true',
        'sysparm_query' : 0
        'sysparm_limit' : fetchSize, 
        'sysparm_offset': offset}